In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras 
import requests
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

tf.__version__

'2.9.2'

In [4]:
pathToTrainData='/content/drive/MyDrive/cs331FinalProject/Experiment/split2/train/'
pathToTestData = '/content/drive/MyDrive/cs331FinalProject/Experiment/split2/test/'

# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(pathToTrainData,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(pathToTestData,
                                            target_size = (224, 224))

Found 5081 images belonging to 20 classes.
Found 1260 images belonging to 20 classes.


In [5]:
url = "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
r = requests.get(url, allow_redirects=True)
open('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'wb').write(r.content)

58889256

In [6]:
# Part 2 - Building the CNN
input_tensor = Input(shape=(224, 224, 3))
vgg_weights = "/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = tf.keras.applications.VGG16(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = None,
)
base_model.load_weights(vgg_weights)
x = base_model.output
print(base_model.input)
print(base_model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [7]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#base_model.trainable = False
x = tf.keras.layers.GlobalAveragePooling2D()
#x = Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(20,activation="softmax")

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
model = Sequential([base_model,
                   x,
                    predictions
                ])
print(model)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 20)                10260     
                                                                 
Total params: 14,724,948
Trainable params: 14,724,948
Non-trainable params: 0
_________________________________________________________________


In [9]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
#base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [11]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
def get_callbacks(name):
    return[
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='val_categorical_crossentropy',patience=1),
    ]

In [13]:
base_learning_rate  = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="categorical_crossentropy",
              metrics=[tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])

In [14]:
history = model.fit(x = training_set, epochs= 20, validation_data = test_set)

Epoch 1/20
159/159 [==============================] - 2258s 14s/step - loss: 2.8701 - categorical_crossentropy: 2.8704 - accuracy: 0.1234 - val_loss: 2.7276 - val_categorical_crossentropy: 2.7284 - val_accuracy: 0.1683
Epoch 2/20
159/159 [==============================] - 121s 759ms/step - loss: 2.6187 - categorical_crossentropy: 2.6187 - accuracy: 0.2395 - val_loss: 2.5067 - val_categorical_crossentropy: 2.5065 - val_accuracy: 0.3190
Epoch 3/20
159/159 [==============================] - 121s 761ms/step - loss: 2.4272 - categorical_crossentropy: 2.4272 - accuracy: 0.3558 - val_loss: 2.3436 - val_categorical_crossentropy: 2.3420 - val_accuracy: 0.4246
Epoch 4/20
159/159 [==============================] - 121s 758ms/step - loss: 2.2774 - categorical_crossentropy: 2.2776 - accuracy: 0.4131 - val_loss: 2.2129 - val_categorical_crossentropy: 2.2139 - val_accuracy: 0.4413
Epoch 5/20
159/159 [==============================] - 123s 772ms/step - loss: 2.1562 - categorical_crossentropy: 2.1562 -

In [15]:
model.evaluate(test_set)

40/40 [==============================] - 15s 362ms/step - loss: 1.4429 - categorical_crossentropy: 1.4370 - accuracy: 0.6230


[1.4428527355194092, 1.4370145797729492, 0.6230158805847168]